In [174]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [ ]:
import time
MAX_ATTEMPTS=3
def fill_table(url,dates,titles,names,dataset_name):
    driver.get(url)# Wait for the page to fully load
    num_pages=driver.find_element_by_xpath("//li[@id='datatable-papers_next']/preceding::li[1]/a").text
    dbtable=soup.find(id='datatable-papers')

    print(dataset_name)
    for i in range(2,int(num_pages)+1):
        print(i)
        #WebDriverWait(driver, 60).until(EC.element_located_to_be_selected((By.XPATH, "//table[@id='datatable-papers']")))
        soup = BeautifulSoup(driver.page_source, 'lxml')
        dbtable=soup.find(id='datatable-papers')
        new_dates=[ d.text for d in dbtable.find_all("div", {"class": 'text-right text-nowrap'})]
        dates += new_dates
        titles += [ a.text for a in dbtable.select('.black-links > a')]
        names += [dataset_name] * len(new_dates)
        attempts=0
        while attempts < MAX_ATTEMPTS:
            try:
                next_page = WebDriverWait(driver, 0).until(EC.visibility_of_element_located((By.XPATH, "//a[@data-dt-idx='{}']".format(i))))      
                '''
                last_height = driver.execute_script("return document.body.scrollHeight")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                driver.execute_script("arguments[0].scrollIntoView();", next_page)
                driver.execute_script("document.body.style.transform='scale(0.9)';")
                '''
                next_page.click()
                break
            except:
                driver.find_element_by_xpath("//li[@id='datatable-papers_previous']/a").click()
                time.sleep(4)
                attempts+=1
        if attempts==MAX_ATTEMPTS: print("ERROR ON ",dataset_name,i,num_pages)
        time.sleep(1)


### Here we'll just get the first 10 papers for every dataset

In [386]:
# Create a new Firefox session

binary = FirefoxBinary('/usr/bin/firefox')
caps = DesiredCapabilities().FIREFOX
caps["marionette"] = True
opts = FirefoxOptions()
opts.add_argument("--headless")
driver = webdriver.Firefox(capabilities=caps, options=opts,firefox_binary=binary, executable_path="/home/bkoch/bin/geckodriver")
driver.implicitly_wait(60)

session = HTMLSession()

pwc_name=[]
pwc_date=[]
pwc_title=[]
pwc_id=[]
pwc_count=[]
is_problematic=[]

In [387]:
with open('datasets.json') as f:
    datasets=json.load(f)   
for i,d in enumerate(datasets):
    #if i<20:continue
    print(d['name'],d['num_papers'])
    driver.get(d['url'])# Wait for the page to fully load
    soup = BeautifulSoup(driver.page_source, 'lxml')
    dataset_id=int(soup.find(id="id_dataset").find_all('option')[1].attrs['value'])
    problem=False
    if driver.page_source.find('flagged-problematic') !=-1: problem=True
    #if soup.find("td", {"class": 'dataTables_empty'}) != None: print(d['name'],"No Papers")
        
    count=0;page=1
    while True:
        url="https://paperswithcode.com/api/internal/papers/?"
        url+=f"paperdataset__dataset_id={dataset_id}&page={page}"
        try: response=json.loads(session.get(url).text)
        except: time.sleep(1)
        pwc_title+=[i['title'] for i in response['results']]
        pwc_date+= [i['publication_date'] for i in response['results']]
        page+=1; count+=len(response['results'])
        if count==int(response['count']): break    
    is_problematic+=[problem]*response['count']
    pwc_name+=[d['name']]*response['count']
    pwc_id+=[dataset_id]*response['count']
    pwc_count+=[response['count']]*response['count']
    pd.DataFrame({'name':pwc_name,'pwc_dataset_id':pwc_id,
                  'title':pwc_title,'date':pwc_date,
                'is_problematic':is_problematic,'paper_count':pwc_count}
                ).to_csv('./datasets_citing_papers.txt',sep='\t',index=False)
    

MNIST 4159
CelebA 1539
JFT-300M 39
GLUE 762
MultiNLI 247
ImageNet 6081
Penn Treebank 1056
WikiText-103 173
LFW 576
WikiSQL 109
OpenAI Gym 653
WikiText-2 173
WikiLarge 25
Food-101 131
Fashion-MNIST 1258
ShapeNet 734
CINIC-10 45
Flickr30k 338
COCO 4139
BSDS500 172
MHP 6
LRW 79
FDDB 140
GOT-10k 79
DukeMTMC-reID 222
KITTI 1590
UCF101 882
HMDB51 431
MIMIC-III 372
TID2013 2
LIVE 1
Visual Genome 502
COCO-Stuff 102
MARS 127
iLIDS-VID 9
Market-1501 533
VIPeR 121
CUHK01 1
PRID2011 13
CUHK03 310
VehicleID 80
BP4D 50
DISFA 84
CUB-200-2011 920
SUN397 3
FewRel 40
DuReader 35
SearchQA 66
CoQA 93
MovieQA 56
Hutter Prize 11
ICDAR 2013 173
Visual Madlibs 11
DAQUAR 34
Visual7W 49
FM-IQA 7
NewsQA 117
TriviaQA 165
RecipeQA 13
SQuAD 873
NarrativeQA 54
CliCR 11
MS MARCO 230
MultiRC 46
HotpotQA 145
RACE 152
QuAC 72
Wizard-of-Oz 21
VCR 51
SWAG 66
Event2Mind 15
XNLI 126
SNLI 643
SciTail 8
CHB-MIT 1
AFW 141
TimeBank 5
SemEval-2010 Task 8 81
IEMOCAP 218
Charades-STA 57
SentEval 69
JFLEG 52
ESC-50 98
Quora Questio

# Everything below here is exploratory junk

In [1]:
d.keys()

NameError: name 'd' is not defined

In [368]:
soup.find(id="id_dataset").find_all('option')[1].attrs['value']

'53'

In [ ]:
titles=[]
dates=[]
ids=[]
url='https://paperswithcode.com/api/internal/papers/?paperdataset__dataset_id=2&page=10'
session = HTMLSession()
with open('datasets.json') as f:
    datasets=json.load(f)   
for i in range(len(datasets)):
    count=0
    page=1
    i=59
    while True:
        url=f"https://paperswithcode.com/api/internal/papers/?paperdataset__dataset_id={i+1}&page={page}"
        response=session.get(url)
        temp=json.loads(response.text)
        titles+=[i['title'] for i in temp['results']]
        dates+= [i['publication_date'] for i in temp['results']]
        print(titles)
        page+=1
        count+=len(temp['results'])
        if count==int(temp['count']): break
    ids += [i+1] * count
    pd.DataFrame({'ids':ids,'title':titles,'date':dates}).to_csv('./papers_citing_datasets_ids.txt',sep='\t',index=False)
    break

In [288]:
url='https://paperswithcode.com/api/internal/papers/?paperdataset__dataset_id=4&page=1'
response=session.get(url)
temp=json.loads(response.text)

In [289]:
temp['count']

0

In [65]:
    pd.DataFrame({'name':names,'title':titles,'date':dates}).to_csv('./papers_citing_datasets.txt',sep='\t',index=False)

In [102]:

url = 'https://paperswithcode.com/dataset/cifar-10'

soup = BeautifulSoup(driver.page_source, 'lxml')


In [99]:
dbtable=soup.find(id='datatable-papers')
soup = BeautifulSoup(driver.page_source, 'lxml')

next_page=driver.find_element_by_xpath("//a[contains(text(),'Next')]")

NameError: name 'soup' is not defined

In [ ]:

        
url = 'https://paperswithcode.com/dataset/cifar-10'
titles=[]
dates=[]
fill_table(url,dates,titles,"CIFAR-10")

In [259]:
temp=json.loads(response.text)
print([i['title'] for i in temp['results']])
print([i['publication_date'] for i in temp['results']])


['Learning Compositional Visual Concepts with Mutual Consistency', 'ELEGANT: Exchanging Latent Encodings with GAN for Transferring Multiple Face Attributes', 'Disentangling Factors of Variation by Mixing Them', 'Generative Adversarial Talking Head: Bringing Portraits to Life with a Weakly Supervised Neural Network', 'DNA-GAN: Learning Disentangled Representations from Multi-Attribute Images', 'Image Inpainting using Block-wise Procedural Training with Annealed Adversarial Counterpart', 'GAGAN: Geometry-Aware Generative Adversarial Networks', 'An Improved Evaluation Framework for Generative Adversarial Networks', 'Super-FAN: Integrated facial landmark localization and super-resolution of real-world low resolution faces in arbitrary poses with GANs', 'Image Semantic Transformation: Faster, Lighter and Stronger']
['2017-11-16', '2018-03-28', '2017-11-20', '2018-03-21', '2017-11-15', '2018-03-23', '2017-12-03', '2018-03-20', '2017-12-07', '2018-03-27']


In [ ]:
temp

In [103]:
driver.get(url)# Wait for the page to fully load

driver.find_element_by_xpath("//a[ddata-dt-idx='2']").click()

In [ ]:
crap=driver.find_element_by_xpath("//a[@data-dt-idx='2']")
print(crap)
BeautifulSoup(driver.page_source, 'lxml')

In [25]:
tables = soup.find_all("div", {"class": 'text-right text-nowrap'})
/html/body/div[11]/div/div[4]/div[1]/div[1]/div[4]/div/div/div[2]/ul/li[9]/a

In [28]:
driver.find_element_by_xpath("//li[@id='datatable-papers_next']/a").click()

In [32]:
driver.find_element_by_xpath("//li[@id='datatable-papers_next']/a").click()
dates = soup.find_all("div", {"class": 'text-right text-nowrap'})
tables

[<div class="text-right text-nowrap">24 May 2018</div>,
 <div class="text-right text-nowrap">21 Jul 2017</div>,
 <div class="text-right text-nowrap">10 Dec 2015</div>,
 <div class="text-right text-nowrap">5 Nov 2016</div>,
 <div class="text-right text-nowrap">2 Dec 2017</div>,
 <div class="text-right text-nowrap">31 Mar 2018</div>,
 <div class="text-right text-nowrap">27 May 2016</div>,
 <div class="text-right text-nowrap">19 Nov 2015</div>,
 <div class="text-right text-nowrap">18 Feb 2020</div>,
 <div class="text-right text-nowrap">6 Feb 2020</div>]

In [52]:
driver.find_element_by_xpath("//li[@id='datatable-papers_next']/preceding-sibling").click()
soup = BeautifulSoup(driver.page_source, 'lxml')
dbtable=soup.find(id='datatable-papers')
dates = [ d.text for d in dbtable.find_all("div", {"class": 'text-right text-nowrap'})]
titles=[ a.text for a in dbtable.select('.black-links > a')]


In [234]:
from requests_html import HTMLSession
session = HTMLSession()

In [ ]:
url='https://paperswithcode.com/api/internal/papers/?paperdataset__dataset_id=2&page=10'
response=session.get(url)
json.loads(response.text)

{'count': 0, 'next': None, 'previous': None, 'results': []}